In [123]:
import pandas as pd
import numpy as np

In [124]:
df = pd.read_csv('salarymodeldata.csv')
df.head()

,Age,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,...,WS/48,OBPM,DBPM,BPM,VORP,MPG,WS/Season,DWS/Season,G/82,PlayerSalary
0,23,1605,30.1,0.619,0.088,0.500,7.4,25.4,16.4,23.3,...,0.253,5.7,1.6,7.3,3.8,37.325581,14.829787,3.489362,75.021277,22471910.0
1,28,1433,29.6,0.618,0.504,0.473,1.5,13.7,7.6,45.4,...,0.287,9.6,0.3,9.9,4.3,35.825000,15.004255,3.140426,69.787234,28299399.0
2,24,1525,28.3,0.630,0.103,0.449,8.1,23.6,16.1,11.6,...,0.248,3.3,1.4,4.7,2.6,36.309524,13.495833,3.587500,71.750000,23775506.0
3,33,1738,28.4,0.628,0.257,0.322,3.6,20.1,12.0,43.1,...,0.211,7.0,2.0,9.0,4.9,36.978723,13.259574,2.965957,82.000000,33285709.0
4,22,1844,24.0,0.633,0.263,0.315,9.5,30.8,20.0,10.2,...,0.214,3.8,1.2,5.0,3.3,35.461538,12.930769,3.626923,82.000000,6216840.0


In [125]:
df = df.loc[df.MP >= 985]

In [126]:
df.columns = df.columns.str.strip()
df.head()

,Age,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,...,WS/48,OBPM,DBPM,BPM,VORP,MPG,WS/Season,DWS/Season,G/82,PlayerSalary
0,23,1605,30.1,0.619,0.088,0.500,7.4,25.4,16.4,23.3,...,0.253,5.7,1.6,7.3,3.8,37.325581,14.829787,3.489362,75.021277,22471910.0
1,28,1433,29.6,0.618,0.504,0.473,1.5,13.7,7.6,45.4,...,0.287,9.6,0.3,9.9,4.3,35.825000,15.004255,3.140426,69.787234,28299399.0
2,24,1525,28.3,0.630,0.103,0.449,8.1,23.6,16.1,11.6,...,0.248,3.3,1.4,4.7,2.6,36.309524,13.495833,3.587500,71.750000,23775506.0
3,33,1738,28.4,0.628,0.257,0.322,3.6,20.1,12.0,43.1,...,0.211,7.0,2.0,9.0,4.9,36.978723,13.259574,2.965957,82.000000,33285709.0
4,22,1844,24.0,0.633,0.263,0.315,9.5,30.8,20.0,10.2,...,0.214,3.8,1.2,5.0,3.3,35.461538,12.930769,3.626923,82.000000,6216840.0


In [127]:
from sklearn.model_selection import train_test_split

X = df.drop('PlayerSalary', axis=1)
y = df['PlayerSalary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [128]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

gb = GradientBoostingRegressor(n_estimators=500, max_depth=4, min_samples_split=2,
                               learning_rate=0.01, loss='ls')
gb.fit(X_train, y_train)

mae = mean_absolute_error(y_train, gb.predict(X_train))
maPe = mae*100.0/y_train.mean()
print(f'${mae:.0f} average error; {maPe:.2f}% error')

$621290 average error; 5.38% error


In [129]:
from sklearn.model_selection import RandomizedSearchCV

# create grid of hyperparameters to search
params = {'loss':['ls', 'lad', 'huber'],
          'learning_rate':[0.1, 0.001],
          'n_estimators':[100, 500, 1000],
          'min_samples_split':[2, 4, 6],
          'min_samples_leaf':[2, 4, 6],
          'max_depth':[5, 10, 20],
          'max_features':['auto', 'log2', 'sqrt']}

model = GradientBoostingRegressor()

random_cv = RandomizedSearchCV(estimator=model,
                               param_distributions=params,
                               cv=5, n_iter=25, n_jobs=-1,
                               scoring='neg_mean_absolute_error')

random_cv.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=25, n_jobs=-1,
          param_distributions={'loss': ['ls', 'lad', 'huber'], 'learning_rate': [0.1, 0.001], 'n_estimators': [100, 500, 1000], 'min_samples_split': [2, 4, 6], 'min_samples_leaf': [2, 4, 6], 'max_depth': [5, 10, 20], 'max_features': ['auto', 'log2', 'sqrt']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=0)

In [130]:
random_cv.best_params_

{'n_estimators': 500,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 20,
 'loss': 'lad',
 'learning_rate': 0.1}

In [131]:
gb = GradientBoostingRegressor(n_estimators=100, max_depth=10, min_samples_split=4,
                               learning_rate=0.1, loss='lad', min_samples_leaf=4,
                               max_features='sqrt')
gb.fit(X_train, y_train)


mae = mean_absolute_error(y_train, gb.predict(X_train))
maPe = mae*100.0/y_train.mean()
print(f'${mae:.0f} average error; {maPe:.2f}% error')

$513207 average error; 4.45% error


In [132]:
gb.fit(X_test, y_test)

mae = mean_absolute_error(y_test, gb.predict(X_test))
maPe = mae*100.0/y_test.mean()
print(f'${mae:.0f} average error; {maPe:.2f}% error')

$1286033 average error; 12.47% error
